# 4. Building Capable Assistant

Key challenge is transforming LLM fluency into reliably capable assistants. This chapter explores methods for instilling greater intelligence, productivity, and trustworthiness in LLMs. The unifying theme across these approaches is enhancing LLMs through prompts, tools, and structured reasoning techniques.

we will begin by addressing the critical weakness of hallucinated content through automatic fact-checking. By verifying claims against the avilable evidence, we can reduce the spread of misinformation. We will continue by discussing a key strength of LLMs with important applications - summarization, which we'll go into with the integration of prompts at different levels of sophistication, and the map reduce approach for very long documents. We will then move on to information extraction from documents with function calls, which leads to the topic of tool integrations. We'll implement an application that showcases how connecting external data and services can augment LLM's limited world knowledge. Finally, we will further extend this application of reasoning strategies.

In short this chapter covers:
* Mitigating hallucinations through fact-checking
* Summarizing information
* Extracting information from documents
* Answering questions with tools
* Exploring reasoning strategies

# 4.1 Mitigating hallucinations through fact-checking
Hallucination in LLMs refers to the generated text being unfaithful or nonsensical compared to the input. It contrasts with faithfulness, where outputs stay consistent with the source. Hallucinations can spread misinformation like disinformation, rumors, and deceptive content. This poses threats to society, including distrust in science, polarization, and democratic processes.

One technique to address hallucinations is automatic fact-checking - verifying claims made by LLMs against evidence from external sources. This allows for catching incorrect or unverified statements.

Fact-checking involves three main stages:
1. __Claim detection__: Identify parts needing verification
2. __Evidence retrieval__: Find sources supporting or refuting the claim
3. __Verdict prediction__: Assess claim veracity based on evidence

Alternative erms for the last two stages are justification production and verdict prediction. We can see the general idea of these stages illustrated in the following diagram (source - https://github.com/Cartus/Automated-Fact-Checking-Resources by Zhijiang Guo):
![](../fig/f4-1.png)

Pre-trained LLMs contain extensive world knowledge that can be prompted for facts. Additionally, external tools can search knowledge bases, Wikipedia, textbooks, and corpora for evidence. By grounding claims in data, fact-checking makes LLMs more reliable.

Pre-trained LLMs contain extensive world knowldge from their training data. Starting with the 24-layer BERT-Large in 2018, language models have been pre-trained on large knowledge bases such as Wikipedia; therefore, they would be able to answer knowledge questions from Wikipedia or - since their training set increasingly includes other sources - the internet, textbooks, arXiv, and GithHub.

We can prompt them with masking and other techniques to retrieve facts for evidence. For example, to answer the question "Where is Microsoft's headqarters located?", the question would be rewritten as "Microsoft's headquarters is in [MASK]" and fedinto a lnaguage model for the answer.

Alternatively, we can integrate external tols to search knowledge bases, Wikipedia, textbooks, and other corpora. The key idea is verifying hallucinated claims by grounding them in factual data sources.

In LangChain, we have a chain available for fact-checking with prompt chaining, where a model actively questions the assumptions that went into a statement. In this self-checking chain, `LLMChecerChain`, the model is prompted sequentially - first, to make the asusmptions explicit, which looks like this:

```
Here's a statement: {statement}\nMake a bullet point list of the
 assumptions you made when proucing the above statement.\n
```

In this string template, where the elements in curly brckets will be replaced by variables. Next, these assumptions are fed back to the model inorder to check them one by one with a prompt like this:

```
Here is a bullet point list of assertions:
    {assertions}
    For each assertion, determine whether it is true or false. If it is false, explain why.\n\n
```
Finally, the model is tasked to make a final judgement:

```
In light of the above facts, how would yu answer the question '{question}'
```
`LLMCheckerChain` does this all by itself, as this example shows:

```python
from langchain.chains import LLMCheckerChain
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.7)
text = "What type of mammal lays the biggest eggs?"
checker_chain = LLMCheckerChain.from_llm(llm, verbose=True)
checker_chain.run(text)
```

In [1]:
from langchain.chains import LLMCheckerChain
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.7)
text = "What type of mammal lays the biggest eggs?"
checker_chain = LLMCheckerChain.from_llm(llm, verbose=True)
checker_chain.run(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...


InvalidRequestError: The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

The model can return different results to this question, some of which are wrong, and some of which it would correctly identify as false.

So, while this technique does not guarantee correct answers, it can put a stop to some incorrect results. Fact-checking approaches involve decomposing claims into smaller checkable queries, which can be formulated as question-answering tasks.

# 4.2 Summarizing information
LLM excel at condensing text through their strong language understanding abilities. We will explore techniques for summarization using LangChain at increasing levels of sophistication.

## 4.2.1 Basic Prompting
For summarizing a couple of sentences, basic prompting works well. Simply instruct the LLM on the desired length and provide a text: 

In [3]:
from langchain import OpenAI
prompt = """
Summarize this text in one sentence: {text}
"""
llm = OpenAI()
summary = llm(prompt.format(text=text))

InvalidRequestError: The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

- `text` is a string variable that can be any text we want to summarize

We can also use the LangChain decorator syntax, which is implemented in the LangChain decorators library, which you should have installed together with all the other dependencies if you followed the instructions in Chapter 3, Getting started with LangChain.

LangChain Decorators provides a more Pythonic interface for defining an executing prompts compared to base LangChain, making it easier to leverage the power of LLMs. Function decorators translate prompt documentation into executable code, enabling multiline definitions and natural code flow.

Here's a decorator example for summarization:



In [4]:
from langchain_decorators import llm_prompt
@llm_prompt
def summarize(text:str, length="short")->str:
    """Summarize this text in {length} length: {text}"""
    return
summary = summarize(text="let me tell you a boring story from when I was young...")

ModuleNotFoundError: No module named 'langchain_core'

The output,the value of the summary variable, I am getting is `The speaker is about a share a story from their youth.`

- `@llm_prompt` decorator translates the docstring into a prompt and handles executing it.
- Parameters are cleanly passed in and outputs are parsed
- Abstraction enables prompting in a natural Python style while handling the complexity behind the scenes, making it easy to focus on creating effective prompts.
By providing this intuitve interface, LangChain Decorators ulock the power for developers.

---
## 4.2.2 Prompt Templates
For dynamic inputs, prompt templates enable inserting text into predefined prompts. Prompt templates allow variable length limits and modular prompt design.

We can implement this in `LangChain Expression Language (LCEL)`:

In [6]:
from langchain import PromptTemplate, OpenAI
from langchain.schema import StrOutputParser
llm = OpenAI()
prompt = PromptTemplate.from_template("Summarize this text: {text}?")
runnable = prompt | llm | StrOutputParser()
summary = runnable.invoke({"text": text})

InvalidRequestError: The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

LCEL provides a declarative way to compose chains that is more intuitive and productive than directly writing code. Key benefits of LCEL include built-in support for asynchronous processing, batching, streaming, fallbacks, parallelism, and seamless integration with LangSmith tracing.

In this case, `runnable` is a chain where the prompt template, the LLM, and the output parser are piped into one another.
---

### 4.2.3 Chain of density
__Chain of Density (CoD)__ to incrementally increase the information density of GPT-4 generated summaries while controlling length. This is the prompt to use with CoD:
![](../fig/f4-2.png)

- Can easily adapt this to any kind of content and provide a different set of guidelines to suit other applications.

- The CoD prompt instructs highly powered LLMs such as GPT-4 to produce an initial sparse, verbose summary of an article containing only a few entities. It then iteratively identifies 1-3 missing entities and fuses them into a rewrite of the previous summary in the same number of words.
- Repeated rewriting under length constraint forces increasing abstraction, fusion of details, and compression to make room for additional entities in each step. The author measure statistics like entity density and source sentence alignment to characterize the desification effects.
- Five iterative steps, summaries become highly condensed with more entities per token packed in through creative rewriting. The authors conduct both human preference studies and GPT-4 scoring to evaluate the impact on overall quality across the density specturm

The results reveal a trade-off between informativeness gained through density and declining coherence from excessive compression. Optimal density balances concision and clarity, with too many entities overwhelming expression. This method and analysis sheds light on controling information density in AI text generation.

---

### 4.2.4 Map-Reduce pipelines
- LangChain supports a `map reduce approach` for processing documents using LLMs, which allows for efficient processing and analysis of documents. A chain can be applied to each document individually and then we combine the outputs into a single document.

- Summarizing long documents can be done by spliting the document into smaller parts (chunks) that are suitable for the token context length of the LLM, and then a map-reduce chain can summarize these chunks independently before recombining.

Key steps:
1. __Map__: Eacj document is passed through a summarization chain (LLM chain).
2. __Collapse__ (optional): The summarized documents are combined into a single document.
3. __Reduce__: The collapsed document go through a final LLM chain to produce the output.

Map step applies a chain to each document in parallel, the reduce step aggregates the mapped outputs and generates the final result.

Optional collapsing, which may involve utilizing LLMs, makes sure the data fits within sequence length limits. Compression steps can be performed recursively.
![](../fig/f4-3.png)

This aproach's implications are that it allows the parallel processing of documents and enables the use of LLMs for reasoning, generating, or analyzing individual documents and combining their outputs.

In [8]:
from langchain.chains.summarize import load_summarize_chain
from langchain import OpenAI
from langchain.document_loaders import PyPDFLoader
pdf_file_path = "<pdf_file_path>"
pdf_loader = PyPDFLoader(pdf_file_path)
docs = pdf_loader.load_and_split()
llm = OpenAI()
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

ValueError: File path <pdf_file_path> is not a valid file or url

- variable pdf_file_path is a string with the path of a PDF file
- Replace the file path with the path to a PDF document.
The default prompt for both the map and reduce steps is this:

In [9]:
"""Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""

'Write a concise summary of the following:\n{text}\nCONCISE SUMMARY:'

- Can specify any prompt for each step
- Text summarization application developed for this chapter on GitHub, we can see how to pass other prompts
On LangChain Hub question-answering-with-sourcesprompt, which takes a reduce/combine prompt like this:

![](../fig/f4-4.png)

In the preceding prompt, we could formulate a concrete question, but equally, we could give the LLM a more abstract instruction to extract assumptions and implications.

The text would be the summaries from the map steps. An instruction like that would help against hallucinations. Other examples of instruction could be _tanslating the document into a different anguage_ or _rephrasing in a certain style_.

By changing the prompt, we can ask any question to be answered from these documents. Thi can be built out into an automation tool that can quickly summarize the content of long texts in a more digestible format, a you should be able to tell from the __summarize__ package in the book's GitHub repository, which show how to focus on different perspectives and structures of the response (adapted from David Shapiro).

* Toughtful prompt engineering with LangChain provides powerful summarization capabilities using LLMs. A few practical tips are:
    * Start with simpler approaches and move to map-reduce if needed
    * Tune chunk size to balance context limits and parallelism
    * Customize map and reduce prompts for the best results
    * Compress or recursively reduce chunks to fit context limits

Once we start making a lot of calls, especially in the _map_ step, if using a cloud provider, tokens will increase thus cost.

---

### 4.2.5 Monitoring token usage
E.g. track the token usage in OpenAI models by hooking into the OpenAI callback:

In [10]:
from langchain import OpenAI, PromptTemplate
from langchain.callbacks import get_openai_callback
llm_chain = PromptTemplate.from_template("Tell me a joke about {topic}!") | OpenAI()
with get_openai_callback() as cb:
    response = llm_chain.invoke(dict(topic="light bulbs"))
    print(response)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

InvalidRequestError: The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

The result will be:

![](../fig/f4-5.png)

The parameters of the model and the prompt can be changed thus the costs and tokenss will change as a consequence.

As an alternative to the OpenAI callback, the `generate()` method of the llm clas returns a response of type `LLMResult` instead of a string. This includes token usages and finish reason, from LangChain docs:

![](../fig/f4-6.png)

The result:

![](../fig/f4-7.png)

Chat completion response format in the OpenAI API includes a usage object with token information.

### 4.3 Extracting Information From Documents
`function calling` builds on instruction tuning, offered by OpenAI api, by describing functions in a schema, developers can tune LLMs to return structured outputs adhering to that schema - for example, extracting entities from text by outputting them in a predefined JSON format.

Function calling enables developers to create chatbots that can answer questions using external tools or OpenAI plugins. It also allows for converting natural language queries into API calls or database queiries and extracting structured data from text. Developers can describe functions to the model using JSON schema and specify the desired function to be called.

In LangChain, we can use these function calls in OpenAI for information extraction, we can obtain specific entities and their properties from a text and their properties from a document in an extraction chain with OpenAI chat models. For example, this can help identify the people mentioned in the text. By using the OpenAI functions parameter and specifying a shcema it ensures that the model outputs the desired entities and properties with their appropriate types.

The implication of this approach are that it allows for precise extraction of entities by defining a schema with the desred properties and their types. It also enables specifying which properties are required and which are optional.

The default format for the schema is a dictionary, but we can also define properties and their types in Pydantic, apopular parsing library, providing control and flexibility in the extraction process.

E.g. a desired schema fro information in a CV:

![](../fig/f4-8.png)


To set up your environment based in Chapter 3 refer back to previous chapter. Also to import the config module here and execute the `set-up_environment()`

Reference: https://github.com/xitanggg/open-resume

![](../fig/f4-9.png)

Utilizing the `create_extraction_chain_pydantic()` function in LangChain, we can provide outschema as input, and an output will be instantiated object that adheres to it. (`pdf_file_path` variable should be the relative/absolute path to a pdf file)

In [13]:
from langchain.chains import create_extraction_chain_pydantic
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
pdf_file_path = "<pdf_file_path>"
pdf_loader = PyPDFLoader(pdf_file_path)
docs = pdf_loader.load_and_split()
# please ote that funciton calling is not enabled for all models!
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613")
chain = create_extraction_chain_pydantic(pydantic_schema=Resume, llm=llm)
chain.run(docs)

ValueError: File path <pdf_file_path> is not a valid file or url

Result...

![](../fig/f4-10.png)

LangChain has natively has the functionality to inject function calls as prompts. This means we can use models from providers other than OpenAI for function calls within LLM apps. We can build this into an interactive webapp with Streamlit.

---

### 4.4 Answering Questions with Tools
LLM are trained on general corpus data and may not be as effective for tasks that require domain-specific knowledge. On their own, LLM can't interact with the environment and access external data sources; however LangChain provides a platform for creating tools that access real-time information and perform tasks such as weather forecasting, making reservations, suggesting recipes, etc. Tools within the framework of agents and chains allow for the development of applications powered by LLMs that are data aware and agentic and open a wide range of approaches to solving problems with LLMs.

### 4.4.1 Information Retrieval with tools
Setup an agent with a few tools:

In [14]:
from langchain.agents import (AgentExecutor, AgentType, initialize_agent, load_tools)
from langchain.chat_models import ChatOpenAI
def load_agent() -> AgentExecutor:
    llm = ChatOpenAI(temperature=0, streaming=True)
    # DuckDuckGoSearchRun, Wolfram alpha, arxiv search, wikipedia
    #TODO: try wolfram-alpha!
    tools = load_tools(tool_names=["ddg-search", "wolfram-alpha", "arxiv", "wikipedia"], llm=llm)
    return initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

This function returns `AgentExecutor`, which is a chain; thus we can integrate into a larger chain. The `Zero-Shot` agent is a general purpose action agent.

Please notice the `streaming` parameter in the ChatOpenAI constructor, which is set to `True`. This makes for a better user experience since it means that the text response will be updated as it comes in, rather than once all the text has been completed.

---

### 4.4.2 Building a Visual Interface
